In [1]:
import requests
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json
import time

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
postcode_path = "./Aust Postcode Location/Australian_Post_Codes_Lat_Lon.csv"

In [4]:
postcode = pd.read_csv(postcode_path)

In [5]:
#Dataframes for postcode long lat list
postcodeDF = pd.DataFrame(postcode)
#Dataframes for final suburbs list
postcodeDF['suburb'] = postcodeDF['suburb'].str.lower()
postcodeDF['type'] = postcodeDF['type'].str.rstrip()
postcodeDF['type'] = postcodeDF['type'].str.lstrip()
postcodeDF

,postcode,suburb,state,dc,type,lat,lon
0,200,australian national university,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,barton,ACT,NaN,LVR,-35.201372,149.095065
2,800,darwin,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,darwin,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,parap,NT,PARAP,Post Office Boxes,-12.432181,130.843310
...,...,...,...,...,...,...,...
16075,9010,brisbane,QLD,CITY DC - BRISBANE,LVR,-27.603479,152.823141
16076,9013,brisbane,QLD,CITY DC - BRISBANE,LVR,-27.603479,152.823141
16077,9015,brisbane,QLD,CITY DC - BRISBANE,LVR,-27.603479,152.823141
16078,9020,brisbane,QLD,CITY DC - BRISBANE,LVR,-27.603479,152.823141


In [6]:
# Get names of indexes for which column state doesn not equal VIC
#indexNames = postcodeDF[ postcodeDF['state'] != "VIC" ].index
vicpostcode_df=postcodeDF[(postcodeDF['state'] == "VIC") & (postcodeDF['type'] == "Delivery Area")]
vicpostcode_df

,postcode,suburb,state,dc,type,lat,lon
5324,3000,melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.814563,144.970267
5326,3002,east melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.816640,144.987811
5327,3003,west melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.806255,144.941123
5328,3004,melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.837324,144.976335
5329,3005,world trade centre,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.822262,144.954856
...,...,...,...,...,...,...,...
8445,3995,wattle bank,VIC,WONTHAGGI,Delivery Area,-38.576309,145.700931
8446,3995,wonthaggi,VIC,WONTHAGGI,Delivery Area,-38.606312,145.588583
8447,3995,woolamai,VIC,WONTHAGGI,Delivery Area,-38.498651,145.520561
8448,3996,inverloch,VIC,INVERLOCH,Delivery Area,-38.632958,145.729641


In [7]:
vic_loc_geo_url="https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json"
response = requests.get(vic_loc_geo_url)

In [8]:
response_json = response.json()

In [9]:
len(response_json["features"])

2973

In [10]:
suburb_list=[]
for loc in response_json["features"]:
    suburb_list.append(loc["properties"]["vic_loca_2"].lower())

In [11]:
len(suburb_list)

2973

In [12]:
vicpostcode_df=postcodeDF[(postcodeDF['state'] == "VIC") & 
                          (postcodeDF['type'] == "Delivery Area") &
                          (postcodeDF['suburb'].isin(suburb_list))]
vicsuburb_df=vicpostcode_df.drop_duplicates(subset=['suburb'])
vicsuburb_df.reset_index(drop=True,inplace=True)
vicsuburb_df

,postcode,suburb,state,dc,type,lat,lon
0,3000,melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.814563,144.970267
1,3002,east melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.816640,144.987811
2,3003,west melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.806255,144.941123
3,3006,southbank,VIC,SOUTH MELBOURNE DC,Delivery Area,-37.823258,144.965926
4,3008,docklands,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.814719,144.948039
...,...,...,...,...,...,...,...
2822,3995,wattle bank,VIC,WONTHAGGI,Delivery Area,-38.576309,145.700931
2823,3995,wonthaggi,VIC,WONTHAGGI,Delivery Area,-38.606312,145.588583
2824,3995,woolamai,VIC,WONTHAGGI,Delivery Area,-38.498651,145.520561
2825,3996,inverloch,VIC,INVERLOCH,Delivery Area,-38.632958,145.729641


In [13]:
vicsuburb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827 entries, 0 to 2826
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   postcode  2827 non-null   int64  
 1   suburb    2827 non-null   object 
 2   state     2827 non-null   object 
 3   dc        2819 non-null   object 
 4   type      2827 non-null   object 
 5   lat       2827 non-null   float64
 6   lon       2827 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 154.7+ KB


In [14]:
#Define path to downloaded Excel for crime data
crimeratepath = "Crime rate/Data_Tables_LGA_Criminal_Incidents_Year_Ending_September_2020.xlsx"

In [15]:
#Create dataframe for crime data - Can take a couple of minutes to read
crimerateDF = pd.read_excel(crimeratepath, sheet_name = "Table 03")
crimerateDF

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2020,September,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B41 Motor vehicle theft,1
1,2020,September,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
2,2020,September,Alpine,3691,Glen Creek,F Other offences,F90 Miscellaneous offences,F93 Cruelty to animals,1
3,2020,September,Alpine,3691,Gundowring,B Property and deception offences,B40 Theft,B49 Other theft,1
4,2020,September,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1
...,...,...,...,...,...,...,...,...,...
315466,2011,September,Yarriambiack,3489,Tempy,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1
315467,2011,September,Yarriambiack,3489,Tempy,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,1
315468,2011,September,Yarriambiack,3489,Tempy,B Property and deception offences,B20 Property damage,B21 Criminal damage,3
315469,2011,September,Yarriambiack,3489,Tempy,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,1


In [16]:
#As suburbs had duplicate postcodes, merging by postcodes caused issues, Determined best to merge via suburbs
#Needed to format crime dataframe in preparation
crimerateDF2 = crimerateDF.rename(columns={"Suburb/Town Name":"suburb","Postcode":"postcode"})
crimerateDF2['suburb'] = crimerateDF2['suburb'].str.lower()
crimerateDF2

,Year,Year ending,Local Government Area,postcode,suburb,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2020,September,Alpine,3691,dederang,B Property and deception offences,B40 Theft,B41 Motor vehicle theft,1
1,2020,September,Alpine,3691,glen creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
2,2020,September,Alpine,3691,glen creek,F Other offences,F90 Miscellaneous offences,F93 Cruelty to animals,1
3,2020,September,Alpine,3691,gundowring,B Property and deception offences,B40 Theft,B49 Other theft,1
4,2020,September,Alpine,3691,gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1
...,...,...,...,...,...,...,...,...,...
315466,2011,September,Yarriambiack,3489,tempy,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1
315467,2011,September,Yarriambiack,3489,tempy,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,1
315468,2011,September,Yarriambiack,3489,tempy,B Property and deception offences,B20 Property damage,B21 Criminal damage,3
315469,2011,September,Yarriambiack,3489,tempy,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,1


In [51]:
crimetype=crimerateDF2[['Offence Division','Offence Subdivision']]
crimetypeDF=crimetype.groupby(['Offence Division','Offence Subdivision']).count()
crimetypeDF

Empty DataFrame
Columns: []
Index: [(A Crimes against the person, A20 Assault and related offences), (A Crimes against the person, A50 Robbery), (A Crimes against the person, A70 Stalking, harassment and threatening behaviour), (A Crimes against the person, A80 Dangerous and negligent acts endangering people), (A Crimes against the person, Other crimes against the person), (B Property and deception offences, B10 Arson), (B Property and deception offences, B20 Property damage), (B Property and deception offences, B30 Burglary/Break and enter), (B Property and deception offences, B40 Theft), (B Property and deception offences, B50 Deception), (B Property and deception offences, B60 Bribery), (C Drug offences, C10 Drug dealing and trafficking), (C Drug offences, C20 Cultivate or manufacture drugs), (C Drug offences, C30 Drug use and possession), (C Drug offences, C90 Other drug offences), (D Public order and security offences, D10 Weapons and explosives offences), (D Public order and security offences, D20 Disorderly and offensive conduct), (D Public order and security offences, D30 Public nuisance offences), (D Public order and security offences, D40 Public security offences), (E Justice procedures offences, E10 Justice procedures), (E Justice procedures offences, E20 Breaches of orders), (F Other offences, F10 Regulatory driving offences), (F Other offences, F20 Transport regulation offences), (F Other offences, F30 Other government regulatory offences), (F Other offences, F90 Miscellaneous offences)]

In [57]:
crimetypeDFreset = crimetypeDF.reset_index(level=['Offence Division', 'Offence Subdivision'])
crimetypeDFreset = crimetypeDFreset[['Offence Subdivision', 'Offence Division']]
crimetypeDFreset

,Offence Subdivision,Offence Division
0,A20 Assault and related offences,A Crimes against the person
1,A50 Robbery,A Crimes against the person
2,"A70 Stalking, harassment and threatening behav...",A Crimes against the person
3,A80 Dangerous and negligent acts endangering p...,A Crimes against the person
4,Other crimes against the person,A Crimes against the person
5,B10 Arson,B Property and deception offences
6,B20 Property damage,B Property and deception offences
7,B30 Burglary/Break and enter,B Property and deception offences
8,B40 Theft,B Property and deception offences
9,B50 Deception,B Property and deception offences


In [17]:
len(crimerateDF2['suburb'].unique())

2825

In [18]:
#To aggregate lowest categetory of offences into nxxt category up, grouping is required
crimegrouping = crimerateDF2.groupby(['Year', 'Local Government Area', 'postcode', 'suburb', 
                                         'Offence Subdivision'])

In [19]:
#Add all related offence subdivisons by year
CrimeSum = crimegrouping['Incidents Recorded'].sum()
CrimeSum

Year  Local Government Area  postcode  suburb         Offence Subdivision                 
2011  Alpine                 3691      dederang       A20 Assault and related offences        1
                                                      B40 Theft                               1
                                                      D20 Disorderly and offensive conduct    1
                                       kancoona       B20 Property damage                     1
                                                      F90 Miscellaneous offences              1
                                                                                             ..
2020  Yarriambiack           3488      speed          A20 Assault and related offences        1
                                                      B50 Deception                           1
                             3491      patchewollock  A20 Assault and related offences        1
                                             

In [20]:
#Convert to dataframe - mult-index
CrimeSumDFindex = pd.DataFrame(CrimeSum)
CrimeSumDFindex

Incidents Recorded
Year Local Government Area postcode suburb        Offence Subdivision                                     
2011 Alpine                3691     dederang      A20 Assault and related offences                       1
                                                  B40 Theft                                              1
                                                  D20 Disorderly and offensive conduct                   1
                                    kancoona      B20 Property damage                                    1
                                                  F90 Miscellaneous offences                             1
...                                                                                                    ...
2020 Yarriambiack          3488     speed         A20 Assault and related offences                       1
                                                  B50 Deception                                          1
                           3491     patchewollock A20 Assault and related offences                       1
                                                  B20 Property damage                                    1
                                                  D10 Weapons and explosives offences                    1

[167490 rows x 1 columns]

In [21]:
#Convert multi-index dataframe to single-index
CrimeSumDFreset = CrimeSumDFindex.reset_index(level=['Year', 'Local Government Area', 'postcode', 
                                         'suburb', 'Offence Subdivision'])
CrimeSumDFreset

,Year,Local Government Area,postcode,suburb,Offence Subdivision,Incidents Recorded
0,2011,Alpine,3691,dederang,A20 Assault and related offences,1
1,2011,Alpine,3691,dederang,B40 Theft,1
2,2011,Alpine,3691,dederang,D20 Disorderly and offensive conduct,1
3,2011,Alpine,3691,kancoona,B20 Property damage,1
4,2011,Alpine,3691,kancoona,F90 Miscellaneous offences,1
...,...,...,...,...,...,...
167485,2020,Yarriambiack,3488,speed,A20 Assault and related offences,1
167486,2020,Yarriambiack,3488,speed,B50 Deception,1
167487,2020,Yarriambiack,3491,patchewollock,A20 Assault and related offences,1
167488,2020,Yarriambiack,3491,patchewollock,B20 Property damage,1


In [22]:
CrimeSumDFPivot=CrimeSumDFreset.pivot(index=['Year', 'Local Government Area', 'postcode', 
                                         'suburb'], columns='Offence Subdivision', values='Incidents Recorded')
CrimeSumDFPivot

Offence Subdivision                                   A20 Assault and related offences  \
Year Local Government Area postcode suburb                                               
2011 Alpine                3691     dederang                                       1.0   
                                    kancoona                                       NaN   
                                    upper gundowring                               NaN   
                           3698     tawonga south                                  NaN   
                           3699     mount beauty                                   1.0   
...                                                                                ...   
2020 Yarriambiack          3480     areegra                                        NaN   
                           3485     woomelang                                      NaN   
                           3487     lascelles                                      1.0   
                           3488     speed                                          1.0   
                           3491     patchewollock                                  1.0   

Offence Subdivision                                   A50 Robbery  \
Year Local Government Area postcode suburb                          
2011 Alpine                3691     dederang                  NaN   
                                    kancoona                  NaN   
                                    upper gundowring          NaN   
                           3698     tawonga south             NaN   
                           3699     mount beauty              NaN   
...                                                           ...   
2020 Yarriambiack          3480     areegra                   NaN   
                           3485     woomelang                 NaN   
                           3487     lascelles                 NaN   
                           3488     speed                     NaN   
                           3491     patchewollock             NaN   

Offence Subdivision                                   A70 Stalking, harassment and threatening behaviour  \
Year Local Government Area postcode suburb                                                                 
2011 Alpine                3691     dederang                                                        NaN    
                                    kancoona                                                        NaN    
                                    upper gundowring                                                NaN    
                           3698     tawonga south                                                   NaN    
                           3699     mount beauty                                                    NaN    
...                                                                                                 ...    
2020 Yarriambiack          3480     areegra                                                         NaN    
                           3485     woomelang                                                       NaN    
                           3487     lascelles                                                       NaN    
                           3488     speed                                                           NaN    
                           3491     patchewollock                                                   NaN    

Offence Subdivision                                   A80 Dangerous and negligent acts endangering people  \
Year Local Government Area postcode suburb                                                                  
2011 Alpine                3691     dederang                                                        NaN     
                                    kancoona                                                        NaN     
                                    upper gundowring                                      

In [23]:
#Convert multi-index dataframe to single-index
CrimeSumDFPivotreset = CrimeSumDFPivot.reset_index(level=['Year', 'Local Government Area', 'postcode', 
                                         'suburb'])
CrimeSumDFPivotreset=CrimeSumDFPivotreset.fillna(0)
CrimeSumDFPivotreset['A20 Assault and related offences']=CrimeSumDFPivotreset['A20 Assault and related offences'].astype(int)
CrimeSumDFPivotreset

Offence Subdivision,Year,Local Government Area,postcode,suburb,A20 Assault and related offences,A50 Robbery,"A70 Stalking, harassment and threatening behaviour",A80 Dangerous and negligent acts endangering people,B10 Arson,B20 Property damage,B30 Burglary/Break and enter,B40 Theft,B50 Deception,B60 Bribery,C10 Drug dealing and trafficking,C20 Cultivate or manufacture drugs,C30 Drug use and possession,C90 Other drug offences,D10 Weapons and explosives offences,D20 Disorderly and offensive conduct,D30 Public nuisance offences,D40 Public security offences,E10 Justice procedures,E20 Breaches of orders,F10 Regulatory driving offences,F20 Transport regulation offences,F30 Other government regulatory offences,F90 Miscellaneous offences,Other crimes against the person
0,2011,Alpine,3691,dederang,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2011,Alpine,3691,kancoona,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2011,Alpine,3691,upper gundowring,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011,Alpine,3698,tawonga south,0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2011,Alpine,3699,mount beauty,1,0.0,0.0,0.0,1.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24201,2020,Yarriambiack,3480,areegra,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24202,2020,Yarriambiack,3485,woomelang,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
24203,2020,Yarriambiack,3487,lascelles,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24204,2020,Yarriambiack,3488,speed,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
CrimeSumDFPivotreset.columns.name = None

In [25]:
CrimeCol=CrimeSumDFPivotreset.columns.tolist()
CrimeList=CrimeCol[4:len(CrimeCol)]
print(CrimeList)
print(len(CrimeList))

['A20 Assault and related offences', 'A50 Robbery', 'A70 Stalking, harassment and threatening behaviour', 'A80 Dangerous and negligent acts endangering people', 'B10 Arson', 'B20 Property damage', 'B30 Burglary/Break and enter', 'B40 Theft', 'B50 Deception', 'B60 Bribery', 'C10 Drug dealing and trafficking', 'C20 Cultivate or manufacture drugs', 'C30 Drug use and possession', 'C90 Other drug offences', 'D10 Weapons and explosives offences', 'D20 Disorderly and offensive conduct', 'D30 Public nuisance offences', 'D40 Public security offences', 'E10 Justice procedures', 'E20 Breaches of orders', 'F10 Regulatory driving offences', 'F20 Transport regulation offences', 'F30 Other government regulatory offences', 'F90 Miscellaneous offences', 'Other crimes against the person']
25


In [26]:
for item in CrimeList:
    CrimeSumDFPivotreset[item]=CrimeSumDFPivotreset[item].astype(int)
CrimeSumDFPivotreset

,Year,Local Government Area,postcode,suburb,A20 Assault and related offences,A50 Robbery,"A70 Stalking, harassment and threatening behaviour",A80 Dangerous and negligent acts endangering people,B10 Arson,B20 Property damage,B30 Burglary/Break and enter,B40 Theft,B50 Deception,B60 Bribery,C10 Drug dealing and trafficking,C20 Cultivate or manufacture drugs,C30 Drug use and possession,C90 Other drug offences,D10 Weapons and explosives offences,D20 Disorderly and offensive conduct,D30 Public nuisance offences,D40 Public security offences,E10 Justice procedures,E20 Breaches of orders,F10 Regulatory driving offences,F20 Transport regulation offences,F30 Other government regulatory offences,F90 Miscellaneous offences,Other crimes against the person
0,2011,Alpine,3691,dederang,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,2011,Alpine,3691,kancoona,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,2011,Alpine,3691,upper gundowring,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2011,Alpine,3698,tawonga south,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2011,Alpine,3699,mount beauty,1,0,0,0,1,3,0,3,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24201,2020,Yarriambiack,3480,areegra,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24202,2020,Yarriambiack,3485,woomelang,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
24203,2020,Yarriambiack,3487,lascelles,1,0,0,0,0,0,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
24204,2020,Yarriambiack,3488,speed,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
CrimeDF=[]
for year in range(2011,2021):
    CrimeDF.append(CrimeSumDFPivotreset[CrimeSumDFPivotreset['Year']==year])

In [28]:
for df in CrimeDF:
    df.drop(columns=['Year'],inplace=True)

C:\Users\QiZheng\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [29]:
CrimeDF[9].dtypes

Local Government Area                                  object
postcode                                                int64
suburb                                                 object
A20 Assault and related offences                        int32
A50 Robbery                                             int32
A70 Stalking, harassment and threatening behaviour      int32
A80 Dangerous and negligent acts endangering people     int32
B10 Arson                                               int32
B20 Property damage                                     int32
B30 Burglary/Break and enter                            int32
B40 Theft                                               int32
B50 Deception                                           int32
B60 Bribery                                             int32
C10 Drug dealing and trafficking                        int32
C20 Cultivate or manufacture drugs                      int32
C30 Drug use and possession                             int32
C90 Othe

In [30]:
vicsuburb_df

,postcode,suburb,state,dc,type,lat,lon
0,3000,melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.814563,144.970267
1,3002,east melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.816640,144.987811
2,3003,west melbourne,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.806255,144.941123
3,3006,southbank,VIC,SOUTH MELBOURNE DC,Delivery Area,-37.823258,144.965926
4,3008,docklands,VIC,CITY DELIVERY CENTRE,Delivery Area,-37.814719,144.948039
...,...,...,...,...,...,...,...
2822,3995,wattle bank,VIC,WONTHAGGI,Delivery Area,-38.576309,145.700931
2823,3995,wonthaggi,VIC,WONTHAGGI,Delivery Area,-38.606312,145.588583
2824,3995,woolamai,VIC,WONTHAGGI,Delivery Area,-38.498651,145.520561
2825,3996,inverloch,VIC,INVERLOCH,Delivery Area,-38.632958,145.729641


In [33]:
CrimeSuburbsDF=vicsuburb_df
CrimeSuburbsDF=CrimeSuburbsDF.merge(CrimeDF[0],how='left',on=['postcode','suburb'],suffixes=('', year))
for idx in range(1,10):
    year=f"_{idx+2011}"
    tmp_df=CrimeDF[idx].drop(columns=['Local Government Area'])
    CrimeSuburbsDF=CrimeSuburbsDF.merge(tmp_df,how='left',on=['postcode','suburb'],suffixes=('', year))
    CrimeSuburbsDF=CrimeSuburbsDF.drop_duplicates(subset=['postcode','suburb'])
CrimeSuburbsDF=CrimeSuburbsDF.fillna(0)
CrimeSuburbsDF

_2012
_2013
_2014
_2015
_2016
_2017
_2018
_2019
_2020


,postcode,suburb,state,dc,type,lat,lon,Local Government Area,A20 Assault and related offences,A50 Robbery,"A70 Stalking, harassment and threatening behaviour",A80 Dangerous and negligent acts endangering people,B10 Arson,B20 Property damage,B30 Burglary/Break and enter,B40 Theft,B50 Deception,B60 Bribery,C10 Drug dealing and trafficking,C20 Cultivate or manufacture drugs,C30 Drug use and possession,C90 Other drug offences,D10 Weapons and explosives offences,D20 Disorderly and offensive conduct,D30 Public nuisance offences,D40 Public security offences,E10 Justice procedures,E20 Breaches of orders,F10 Regulatory driving offences,F20 Transport regulation offences,F30 Other government regulatory offences,F90 Miscellaneous offences,Other crimes against the person,A20 Assault and related offences_2012,A50 Robbery_2012,"A70 Stalking, harassment and threatening behaviour_2012",A80 Dangerous and negligent acts endangering people_2012,B10 Arson_2012,B20 Property damage_2012,B30 Burglary/Break and enter_2012,B40 Theft_2012,B50 Deception_2012,B60 Bribery_2012,C10 Drug dealing and trafficking_2012,C20 Cultivate or manufacture drugs_2012,C30 Drug use and possession_2012,C90 Other drug offences_2012,D10 Weapons and explosives offences_2012,D20 Disorderly and offensive conduct_2012,D30 Public nuisance offences_2012,D40 Public security offences_2012,E10 Justice procedures_2012,E20 Breaches of orders_2012,F10 Regulatory driving offences_2012,F20 Transport regulation offences_2012,F30 Other government regulatory offences_2012,F90 Miscellaneous offences_2012,Other crimes against the person_2012,A20 Assault and related offences_2013,A50 Robbery_2013,"A70 Stalking, harassment and threatening behaviour_2013",A80 Dangerous and negligent acts endangering people_2013,B10 Arson_2013,B20 Property damage_2013,B30 Burglary/Break and enter_2013,B40 Theft_2013,B50 Deception_2013,B60 Bribery_2013,C10 Drug dealing and trafficking_2013,C20 Cultivate or manufacture drugs_2013,C30 Drug use and possession_2013,C90 Other drug offences_2013,D10 Weapons and explosives offences_2013,D20 Disorderly and offensive conduct_2013,D30 Public nuisance offences_2013,D40 Public security offences_2013,E10 Justice procedures_2013,E20 Breaches of orders_2013,F10 Regulatory driving offences_2013,F20 Transport regulation offences_2013,F30 Other government regulatory offences_2013,F90 Miscellaneous offences_2013,Other crimes against the person_2013,A20 Assault and related offences_2014,A50 Robbery_2014,"A70 Stalking, harassment and threatening behaviour_2014",A80 Dangerous and negligent acts endangering people_2014,B10 Arson_2014,B20 Property damage_2014,B30 Burglary/Break and enter_2014,B40 Theft_2014,B50 Deception_2014,B60 Bribery_2014,C10 Drug dealing and trafficking_2014,C20 Cultivate or manufacture drugs_2014,C30 Drug use and possession_2014,C90 Other drug offences_2014,D10 Weapons and explosives offences_2014,D20 Disorderly and offensive conduct_2014,D30 Public nuisance offences_2014,D40 Public security offences_2014,E10 Justice procedures_2014,E20 Breaches of orders_2014,F10 Regulatory driving offences_2014,F20 Transport regulation offences_2014,F30 Other government regulatory offences_2014,F90 Miscellaneous offences_2014,Other crimes against the person_2014,A20 Assault and related offences_2015,A50 Robbery_2015,"A70 Stalking, harassment and threatening behaviour_2015",A80 Dangerous and negligent acts endangering people_2015,B10 Arson_2015,B20 Property damage_2015,B30 Burglary/Break and enter_2015,B40 Theft_2015,B50 Deception_2015,B60 Bribery_2015,C10 Drug dealing and trafficking_2015,C20 Cultivate or manufacture drugs_2015,C30 Drug use and possession_2015,C90 Other drug offences_2015,D10 Weapons and explosives offences_2015,D20 Disorderly and offensive conduct_2015,D30 Public nuisance offences_2015,D40 Public security offences_2015,E10 Justice procedures_2015,E20 Breaches of orders_2015,F10 Regulatory driving offences_2015,F20 Transport regulation offences_2015,F30 Other gove

In [34]:
for item in CrimeList:
    new_name=f"{item}_2011"
    CrimeSuburbsDF = CrimeSuburbsDF.rename(columns={item:new_name})

In [35]:
CrimeSubCol=CrimeSuburbsDF.columns.tolist()
CrimeSubList=CrimeSubCol[8:len(CrimeSubCol)]
for item in CrimeSubList:
    CrimeSuburbsDF[item]=CrimeSuburbsDF[item].astype(int)
CrimeSuburbsDF

,postcode,suburb,state,dc,type,lat,lon,Local Government Area,A20 Assault and related offences_2011,A50 Robbery_2011,"A70 Stalking, harassment and threatening behaviour_2011",A80 Dangerous and negligent acts endangering people_2011,B10 Arson_2011,B20 Property damage_2011,B30 Burglary/Break and enter_2011,B40 Theft_2011,B50 Deception_2011,B60 Bribery_2011,C10 Drug dealing and trafficking_2011,C20 Cultivate or manufacture drugs_2011,C30 Drug use and possession_2011,C90 Other drug offences_2011,D10 Weapons and explosives offences_2011,D20 Disorderly and offensive conduct_2011,D30 Public nuisance offences_2011,D40 Public security offences_2011,E10 Justice procedures_2011,E20 Breaches of orders_2011,F10 Regulatory driving offences_2011,F20 Transport regulation offences_2011,F30 Other government regulatory offences_2011,F90 Miscellaneous offences_2011,Other crimes against the person_2011,A20 Assault and related offences_2012,A50 Robbery_2012,"A70 Stalking, harassment and threatening behaviour_2012",A80 Dangerous and negligent acts endangering people_2012,B10 Arson_2012,B20 Property damage_2012,B30 Burglary/Break and enter_2012,B40 Theft_2012,B50 Deception_2012,B60 Bribery_2012,C10 Drug dealing and trafficking_2012,C20 Cultivate or manufacture drugs_2012,C30 Drug use and possession_2012,C90 Other drug offences_2012,D10 Weapons and explosives offences_2012,D20 Disorderly and offensive conduct_2012,D30 Public nuisance offences_2012,D40 Public security offences_2012,E10 Justice procedures_2012,E20 Breaches of orders_2012,F10 Regulatory driving offences_2012,F20 Transport regulation offences_2012,F30 Other government regulatory offences_2012,F90 Miscellaneous offences_2012,Other crimes against the person_2012,A20 Assault and related offences_2013,A50 Robbery_2013,"A70 Stalking, harassment and threatening behaviour_2013",A80 Dangerous and negligent acts endangering people_2013,B10 Arson_2013,B20 Property damage_2013,B30 Burglary/Break and enter_2013,B40 Theft_2013,B50 Deception_2013,B60 Bribery_2013,C10 Drug dealing and trafficking_2013,C20 Cultivate or manufacture drugs_2013,C30 Drug use and possession_2013,C90 Other drug offences_2013,D10 Weapons and explosives offences_2013,D20 Disorderly and offensive conduct_2013,D30 Public nuisance offences_2013,D40 Public security offences_2013,E10 Justice procedures_2013,E20 Breaches of orders_2013,F10 Regulatory driving offences_2013,F20 Transport regulation offences_2013,F30 Other government regulatory offences_2013,F90 Miscellaneous offences_2013,Other crimes against the person_2013,A20 Assault and related offences_2014,A50 Robbery_2014,"A70 Stalking, harassment and threatening behaviour_2014",A80 Dangerous and negligent acts endangering people_2014,B10 Arson_2014,B20 Property damage_2014,B30 Burglary/Break and enter_2014,B40 Theft_2014,B50 Deception_2014,B60 Bribery_2014,C10 Drug dealing and trafficking_2014,C20 Cultivate or manufacture drugs_2014,C30 Drug use and possession_2014,C90 Other drug offences_2014,D10 Weapons and explosives offences_2014,D20 Disorderly and offensive conduct_2014,D30 Public nuisance offences_2014,D40 Public security offences_2014,E10 Justice procedures_2014,E20 Breaches of orders_2014,F10 Regulatory driving offences_2014,F20 Transport regulation offences_2014,F30 Other government regulatory offences_2014,F90 Miscellaneous offences_2014,Other crimes against the person_2014,A20 Assault and related offences_2015,A50 Robbery_2015,"A70 Stalking, harassment and threatening behaviour_2015",A80 Dangerous and negligent acts endangering people_2015,B10 Arson_2015,B20 Property damage_2015,B30 Burglary/Break and enter_2015,B40 Theft_2015,B50 Deception_2015,B60 Bribery_2015,C10 Drug dealing and trafficking_2015,C20 Cultivate or manufacture drugs_2015,C30 Drug use and possession_2015,C90 Other drug offences_2015,D10 Weapons and explosives offences_2015,D20 Disorderly and offensive conduct_2015,D30 Public nuisance offences_2015,D40 Public security offences_2015,E10 Justice procedure

In [60]:
crimetypeDFreset.to_csv('CrimeTypes.csv',index=False)

In [58]:
CrimeSuburbsDF.to_csv('CrimeSuburbs.csv',index=False)

In [41]:
CrimeSuburbsDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2827 entries, 0 to 2963
Columns: 258 entries, postcode to Other crimes against the person_2020
dtypes: float64(2), int32(250), int64(1), object(5)
memory usage: 2.9+ MB
